In [1]:
%%time
# Define a Function to build a model and predict a list of recommended songs.
def songRecommender(track_id, mongoDB, traits_dictionary, num_recs):
    import pandas as pd
    import os
    from sklearn.neighbors import NearestNeighbors
    if num_recs == "" or num_recs == None:
      num_recs = 10
    
    # Convert DataBased Songs into a DataFram
    original_df = pd.json_normalize(mongoDB)
    original_df = original_df[original_df['track_id'] != track_id].reset_index(drop=True)

    # Extract Song Qualities
    song_q = original_df.drop(columns=['track_id', 'artists', 'album_name', 'track_name', 'track_genre', 'duration_ms', 'track_url', 'popularity', 'explicit','_id']).dropna().reset_index(drop=True)
    
    # print(song_q.info())

    # Establish KNN Model
    knn = NearestNeighbors(n_neighbors=int(num_recs))
    knn.fit(song_q)

    # Convert Searched Song Traits
    test = pd.DataFrame(traits_dictionary)
    test = test.drop(columns=['track_id', 'artists', 'album_name', 'track_name', 'track_genre', 'duration_ms', 'track_url', 'popularity', 'explicit', '_id'])
    
    # Find Similar Songs from KNN Model
    result = knn.kneighbors(test)[1].tolist()[0]
    
    # Filter DataBase Songs by Result Indices
    result_df = original_df.iloc[result]
    
    # Create Clickable Links & Clean Artists
    for index, track in result_df.iterrows():
      link = track['track_url']
      name = track['track_name']
      result_df.loc[index,'track_name'] = f'<a href={link}>{name}</a>'
      arts = track['artists'].strip("][").split(",")
      result_df.loc[index, 'artists'] = ", ".join(arts).replace("'","")
    
    return result_df[['track_name', 'artists', 'track_genre']]

CPU times: user 611 ms, sys: 137 ms, total: 748 ms
Wall time: 752 ms


/Users/nburwick/anaconda3/lib/python3.9/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


In [2]:
result_df

,track_name,artists,track_url
67385,Санкт - Петербург (все звёзды),['Александр Добронравов'],NaN
17317,In My Blood,['Shawn Mendes'],https://open.spotify.com/track/2QZ7WLBE8h2y1Y5...
53389,忽然很想你,['許慧欣'],NaN
44713,Watching TV,['Sara Kays'],NaN
10228,脆弱,['Kay Tse'],https://open.spotify.com/track/01IVFofCRf1pyda...
...,...,...,...
61510,Story of My Life,['One Direction'],NaN
2693,abcdefu,['GAYLE'],https://open.spotify.com/track/4fouWK6XVHhzl78...
61410,august,['Taylor Swift'],NaN
76032,Everybody Wants To Rule The World,['Tears For Fears'],NaN
